In [ ]:
# load libraries
import math
import pandas as pd
import numpy as np
from keras.layers.recurrent import GRU
from keras.layers.core import Dense, Dropout
from keras.models import Sequential
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# load the dataset
df = pd.read_csv("~\\data1.csv", header=0,engine='python') 

In [ ]:
df = df.loc[0:117]
df = df[["날짜","정책","정책지수","누적정책지수","서울","용산구","광진구","서대문구","영등포구","서초구","소비자물가지수","한국은행기준금리","원달러환율","가계대출","서울시부동산심리지수","서울실거래지수","도심권실거래지수","동북권실거래지수","서북권실거래지수","서남권실거래지수","동남권실거래지수"]]
df.columns = ["date","1","PI","PI2","Seoul","Yongsan-gu","Gwangjin-gu","Seodaemun-gu","Yeongdeungpo-gu","Seocho-gu","CPI","BR","ER","HL","EI","SI","CI","NEI","NWI","SWI","SEI"]
df.tail()

In [ ]:
feature = df[["PI","CPI","BR","ER","HL","EI"]]

whole1 = df[["Seoul","PI","CPI","BR","ER","HL","EI"]]

whole2 = df[["Yongsan-gu","PI","CPI","BR","ER","HL","EI"]]
whole3 = df[["Gwangjin-gu","PI","CPI","BR","ER","HL","EI"]]
whole4 = df[["Seodaemun-gu","PI","CPI","BR","ER","HL","EI"]]
whole5 = df[["Yeongdeungpo-gu","PI","CPI","BR","ER","HL","EI"]]
whole6 = df[["Seocho-gu","CPI","PI","BR","ER","HL","EI"]]

label = df[["Seoul","Yongsan-gu","Gwangjin-gu","Seodaemun-gu","Yeongdeungpo-gu","Seocho-gu"]]

label1 = df[["Seoul"]]
label2 = df[["Yongsan-gu"]]
label3 = df[["Gwangjin-gu"]]
label4 = df[["Seodaemun-gu"]]
label5 = df[["Yeongdeungpo-gu"]]
label6 = df[["Seocho-gu"]]

# 1. Yongsan-gu

In [ ]:
# normalize the dataset
scaler = MinMaxScaler()
xdata = scaler.fit_transform(feature)
ydata = scaler.fit_transform(label2)
wholedata=scaler.fit_transform(whole2)

In [ ]:
# Window -> X timestep back
step_back = 1
X_whole, Y_whole = [], []
for i in range(len(wholedata)-step_back - 1):
    a = wholedata[i:(i+step_back), 1:]
    X_whole.append(a)
    Y_whole.append(wholedata[i + step_back, 0])
X_whole = np.array(X_whole); Y_whole = np.array(Y_whole)
X_whole.shape

In [ ]:
from sklearn.model_selection import train_test_split,KFold,cross_val_score
import keras.backend as K 
import tensorflow as tf

kfold=KFold(10)

K.clear_session()

cvscores = []

for train, test in kfold.split(X_whole, Y_whole):

    X_train, X_test, y_train, y_test = train_test_split(X_whole, Y_whole, test_size = 0.1)
    X_train = np.reshape(X_train, (X_train.shape[0], step_back, 6))
    X_test  = np.reshape(X_test, (X_test.shape[0], step_back, 6))
    
    output_size=7
    activ_func="relu"
    dropout=0.25
    loss="mean_squared_error" 
    
    model = Sequential()
    model.add(GRU(75, return_sequences=True,input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(10))
    model.add(Dropout(dropout))
    model.add(GRU(units=30, return_sequences=True))
    model.add(Dense(units=output_size,activation=activ_func))

    model.compile(loss=loss, optimizer='adam',metrics =["mse"])

    model.fit(X_train, y_train, epochs=50, batch_size=24, verbose=0,shuffle=False)
    
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [ ]:
# split into train and test sets
train_size = int(len(xdata)*0.9)+4
x_train_dataset, x_test_dataset = xdata[0:train_size,:], xdata[train_size:,:]
y_train_dataset, y_test_dataset = ydata[0:train_size], ydata[train_size:]

# Window -> X timestep back
step_back= 1
X_train, Y_train = [], []
for i in range(len(x_train_dataset)-step_back - 1):
    a = x_train_dataset[i:(i+step_back), :]
    X_train.append(a)
    Y_train.append(y_train_dataset[i + step_back, 0])
X_train = np.array(X_train); Y_train = np.array(Y_train)
    
X_test, Y_test = [], []
for i in range(len(x_test_dataset)-step_back - 1):
    a = x_test_dataset[i:(i+step_back), :]
    X_test.append(a)
    Y_test.append(y_test_dataset[i + step_back, 0])
X_test = np.array(X_test); Y_test = np.array(Y_test)

print(X_train.shape); print(Y_train.shape);             
print(X_test.shape); print(Y_test.shape)


In [ ]:
# reshape input to be [samples, time steps, features]
    
X_train = np.reshape(X_train, (X_train.shape[0], step_back, 6))
X_test  = np.reshape(X_test, (X_test.shape[0], step_back, 6))
Y_train = np.reshape(Y_train, (Y_train.shape[0], 1))
Y_test  = np.reshape(Y_test, (Y_test.shape[0], 1))

In [ ]:
# setup a GRU network in keras

output_size=7
activ_func="relu"
dropout=0.25
loss="mean_squared_error" 

model_gru = Sequential()
model_gru.add(GRU(75, return_sequences=True,input_shape=(X_train.shape[1], X_train.shape[2])))
model_gru.add(Dense(10))
model_gru.add(Dropout(0.25))
model_gru.add(GRU(units=30, return_sequences=True))
model_gru.add(Dense(units=output_size, activation="relu"))

model_gru.compile(loss='mean_squared_error', optimizer='adam')
model_gru.summary()

In [ ]:
gru_history = model_gru.fit(X_train, Y_train, epochs=50, batch_size=24, validation_data=(X_test, Y_test), shuffle=False, verbose=1)

In [ ]:
# Estimate model performance
trainScore = model_gru.evaluate(X_train, Y_train, verbose=1)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, np.sqrt(trainScore)))

In [ ]:
# Evaluate the skill of the Trained model
testPredict  = model_gru.predict(X_test)

In [ ]:
testPredict.shape

In [ ]:
result1=[]
for i in range(0,testPredict.shape[0]):
    result1.append(testPredict[i][0])
    i=+1
result1

In [ ]:
# invert predictions
testPredict1 = scaler.inverse_transform(result1)

In [ ]:
result_pred=[]
for i in range(0,testPredict1.shape[0]):
    result_pred.append(testPredict1[i][0])
    i=+1

result_pred

In [ ]:
result11=[]
for i in range(0,len(result1)):
    result11.append(result1[i][0])
    i=+1
result11=np.reshape(result11,(-1,1))

In [ ]:
#rmse
math.sqrt(sum((result11-Y_test)**2)/6)

In [ ]:
#plt.figure(figsize=(10,6))
plt.scatter(result11,Y_test)

In [ ]:
# plot baseline and predictions
#plt.figure(figsize=(10,6))
plt.plot(result_pred)
plt.plot(label2[-6:].values)
plt.show()

# 2. Gwangjin-gu

In [ ]:
# normalize the dataset
scaler = MinMaxScaler()
xdata = scaler.fit_transform(feature)
ydata = scaler.fit_transform(label3)
wholedata=scaler.fit_transform(whole3)

In [ ]:
# Window -> X timestep back
step_back = 1
X_whole, Y_whole = [], []
for i in range(len(wholedata)-step_back - 1):
    a = wholedata[i:(i+step_back), 1:]
    X_whole.append(a)
    Y_whole.append(wholedata[i + step_back, 0])
X_whole = np.array(X_whole); Y_whole = np.array(Y_whole)
X_whole.shape

In [ ]:
from sklearn.model_selection import train_test_split,KFold,cross_val_score
import keras.backend as K 
import tensorflow as tf

kfold=KFold(10)

K.clear_session()

cvscores = []

for train, test in kfold.split(X_whole, Y_whole):

    X_train, X_test, y_train, y_test = train_test_split(X_whole, Y_whole, test_size = 0.1)
    X_train = np.reshape(X_train, (X_train.shape[0], step_back, 6))
    X_test  = np.reshape(X_test, (X_test.shape[0], step_back, 6))
    
    output_size=7
    activ_func="relu"
    dropout=0.25
    loss="mean_squared_error" 
    
    model = Sequential()
    model.add(GRU(75, return_sequences=True,input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(10))
    model.add(Dropout(dropout))
    model.add(GRU(units=30, return_sequences=True))
    model.add(Dense(units=output_size,activation=activ_func))

    model.compile(loss=loss, optimizer='adam',metrics =["mse"])

    model.fit(X_train, y_train, epochs=70, batch_size=12, verbose=0,shuffle=False)
    
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [ ]:
# split into train and test sets
train_size = int(len(xdata)*0.9)+4
x_train_dataset, x_test_dataset = xdata[0:train_size,:], xdata[train_size:,:]
y_train_dataset, y_test_dataset = ydata[0:train_size], ydata[train_size:]

In [ ]:
# Window -> X timestep back
step_back= 1
X_train, Y_train = [], []
for i in range(len(x_train_dataset)-step_back - 1):
    a = x_train_dataset[i:(i+step_back), :]
    X_train.append(a)
    Y_train.append(y_train_dataset[i + step_back, 0])
X_train = np.array(X_train); Y_train = np.array(Y_train)
    
X_test, Y_test = [], []
for i in range(len(x_test_dataset)-step_back - 1):
    a = x_test_dataset[i:(i+step_back), :]
    X_test.append(a)
    Y_test.append(y_test_dataset[i + step_back, 0])
X_test = np.array(X_test); Y_test = np.array(Y_test)

print(X_train.shape); print(Y_train.shape);             
print(X_test.shape); print(Y_test.shape)

In [ ]:
# reshape input to be [samples, time steps, features]
    
X_train = np.reshape(X_train, (X_train.shape[0], step_back, 6))
X_test  = np.reshape(X_test, (X_test.shape[0], step_back, 6))
Y_train = np.reshape(Y_train, (Y_train.shape[0], 1))
Y_test  = np.reshape(Y_test, (Y_test.shape[0], 1))

In [ ]:
# setup a GRU network in keras

output_size=7
activ_func="relu"
dropout=0.25
loss="mean_squared_error" 

model_gru = Sequential()
model_gru.add(GRU(75, return_sequences=True,input_shape=(X_train.shape[1], X_train.shape[2])))
model_gru.add(Dense(10))
model_gru.add(Dropout(0.25))
model_gru.add(GRU(units=30, return_sequences=True))
model_gru.add(Dense(units=output_size, activation="relu"))

model_gru.compile(loss='mean_squared_error', optimizer='adam')
model_gru.summary()

In [ ]:
gru_history = model_gru.fit(X_train, Y_train, epochs=70, batch_size=12, validation_data=(X_test, Y_test), shuffle=False, verbose=1)

In [ ]:
# Estimate model performance
trainScore = model_gru.evaluate(X_train, Y_train, verbose=1)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, np.sqrt(trainScore)))

In [ ]:
# Evaluate the skill of the Trained model
testPredict  = model_gru.predict(X_test)
testPredict.shape

In [ ]:
result1=[]
for i in range(0,testPredict.shape[0]):
    result1.append(testPredict[i][0])
    i=+1
result1

In [ ]:
# invert predictions
testPredict1 = scaler.inverse_transform(result1)

In [ ]:
result_pred=[]
for i in range(0,testPredict1.shape[0]):
    result_pred.append(testPredict1[i][0])
    i=+1

result_pred

In [ ]:
result11=[]
for i in range(0,len(result1)):
    result11.append(result1[i][0])
    i=+1
result11=np.reshape(result11,(-1,1))

In [ ]:
#rmse
math.sqrt(sum((result11-Y_test)**2)/6)

In [ ]:
#plt.figure(figsize=(10,6))
plt.scatter(result11,Y_test)

In [ ]:
# plot baseline and predictions
#plt.figure(figsize=(10,6))
plt.plot(result_pred)
plt.plot(label3[-6:].values)
plt.show()

# 3. Seodaemun-gu

In [ ]:
# normalize the dataset
scaler = MinMaxScaler()
xdata = scaler.fit_transform(feature)
ydata = scaler.fit_transform(label4)
wholedata=scaler.fit_transform(whole4)

In [ ]:
# Window -> X timestep back
step_back = 1
X_whole, Y_whole = [], []
for i in range(len(wholedata)-step_back - 1):
    a = wholedata[i:(i+step_back), 1:]
    X_whole.append(a)
    Y_whole.append(wholedata[i + step_back, 0])
X_whole = np.array(X_whole); Y_whole = np.array(Y_whole)
X_whole.shape

In [ ]:
from sklearn.model_selection import train_test_split,KFold,cross_val_score
import keras.backend as K 
import tensorflow as tf

kfold=KFold(10)

K.clear_session()

cvscores = []

for train, test in kfold.split(X_whole, Y_whole):

    X_train, X_test, y_train, y_test = train_test_split(X_whole, Y_whole, test_size = 0.1)
    X_train = np.reshape(X_train, (X_train.shape[0], step_back, 6))
    X_test  = np.reshape(X_test, (X_test.shape[0], step_back, 6))
    
    output_size=7
    activ_func="relu"
    dropout=0.25
    loss="mean_squared_error" 
    
    model = Sequential()
    model.add(GRU(75, return_sequences=True,input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(10))
    model.add(Dropout(dropout))
    model.add(GRU(units=30, return_sequences=True))
    model.add(Dense(units=output_size,activation=activ_func))

    model.compile(loss=loss, optimizer='adam',metrics =["mse"])

    model.fit(X_train, y_train, epochs=50, batch_size=24, verbose=0,shuffle=False)
    
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [ ]:
# split into train and test sets
train_size = int(len(xdata)*0.9)+4
x_train_dataset, x_test_dataset = xdata[0:train_size,:], xdata[train_size:,:]
y_train_dataset, y_test_dataset = ydata[0:train_size], ydata[train_size:]

In [ ]:
# Window -> X timestep back
step_back= 1
X_train, Y_train = [], []
for i in range(len(x_train_dataset)-step_back - 1):
    a = x_train_dataset[i:(i+step_back), :]
    X_train.append(a)
    Y_train.append(y_train_dataset[i + step_back, 0])
X_train = np.array(X_train); Y_train = np.array(Y_train)
    
X_test, Y_test = [], []
for i in range(len(x_test_dataset)-step_back - 1):
    a = x_test_dataset[i:(i+step_back), :]
    X_test.append(a)
    Y_test.append(y_test_dataset[i + step_back, 0])
X_test = np.array(X_test); Y_test = np.array(Y_test)

print(X_train.shape); print(Y_train.shape);             
print(X_test.shape); print(Y_test.shape)

In [ ]:
# reshape input to be [samples, time steps, features]
    
X_train = np.reshape(X_train, (X_train.shape[0], step_back, 6))
X_test  = np.reshape(X_test, (X_test.shape[0], step_back, 6))
Y_train = np.reshape(Y_train, (Y_train.shape[0], 1))
Y_test  = np.reshape(Y_test, (Y_test.shape[0], 1))

In [ ]:
# setup a GRU network in keras

output_size=7
activ_func="relu"
dropout=0.25
loss="mean_squared_error" 

model_gru = Sequential()
model_gru.add(GRU(75, return_sequences=True,input_shape=(X_train.shape[1], X_train.shape[2])))
model_gru.add(Dense(10))
model_gru.add(Dropout(0.25))
model_gru.add(GRU(units=30, return_sequences=True))
model_gru.add(Dense(units=output_size, activation="relu"))

model_gru.compile(loss='mean_squared_error', optimizer='adam')
model_gru.summary()

In [ ]:
gru_history = model_gru.fit(X_train, Y_train, epochs=50, batch_size=24, validation_data=(X_test, Y_test), shuffle=False, verbose=1)

In [ ]:
# Estimate model performance
trainScore = model_gru.evaluate(X_train, Y_train, verbose=1)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, np.sqrt(trainScore)))

In [ ]:
# Evaluate the skill of the Trained model
testPredict  = model_gru.predict(X_test)
testPredict.shape

In [ ]:
result1=[]
for i in range(0,testPredict.shape[0]):
    result1.append(testPredict[i][0])
    i=+1
result1

In [ ]:
# invert predictions
testPredict1 = scaler.inverse_transform(result1)

In [ ]:
result_pred=[]
for i in range(0,testPredict1.shape[0]):
    result_pred.append(testPredict1[i][0])
    i=+1

result_pred

In [ ]:
result11=[]
for i in range(0,len(result1)):
    result11.append(result1[i][0])
    i=+1
result11=np.reshape(result11,(-1,1))

In [ ]:
#rmse
math.sqrt(sum((result11-Y_test)**2)/6)

In [ ]:
#plt.figure(figsize=(10,6))
plt.scatter(result11,Y_test)

In [ ]:
# plot baseline and predictions
#plt.figure(figsize=(10,6))
plt.plot(result_pred)
plt.plot(label4[-6:].values)
plt.show()

# 4. Yeongdeungpo-gu

In [ ]:
# normalize the dataset
scaler = MinMaxScaler()
xdata = scaler.fit_transform(feature)
ydata = scaler.fit_transform(label5)
wholedata=scaler.fit_transform(whole5)

In [ ]:
# Window -> X timestep back
step_back = 1
X_whole, Y_whole = [], []
for i in range(len(wholedata)-step_back - 1):
    a = wholedata[i:(i+step_back), 1:]
    X_whole.append(a)
    Y_whole.append(wholedata[i + step_back, 0])
X_whole = np.array(X_whole); Y_whole = np.array(Y_whole)
X_whole.shape

In [ ]:
from sklearn.model_selection import train_test_split,KFold,cross_val_score
import keras.backend as K 
import tensorflow as tf

kfold=KFold(10)

K.clear_session()

cvscores = []

for train, test in kfold.split(X_whole, Y_whole):

    X_train, X_test, y_train, y_test = train_test_split(X_whole, Y_whole, test_size = 0.1)
    X_train = np.reshape(X_train, (X_train.shape[0], step_back, 6))
    X_test  = np.reshape(X_test, (X_test.shape[0], step_back, 6))
    
    output_size=7
    activ_func="relu"
    dropout=0.25
    loss="mean_squared_error" 
    
    model = Sequential()
    model.add(GRU(75, return_sequences=True,input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(5))
    model.add(Dropout(dropout))
    model.add(GRU(units=30, return_sequences=True))
    model.add(Dense(units=output_size,activation=activ_func))

    model.compile(loss=loss, optimizer='adam',metrics =["mse"])

    model.fit(X_train, y_train, epochs=70, batch_size=12, verbose=0,shuffle=False)
    
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [ ]:
# split into train and test sets
train_size = int(len(xdata)*0.9)+4
x_train_dataset, x_test_dataset = xdata[0:train_size,:], xdata[train_size:,:]
y_train_dataset, y_test_dataset = ydata[0:train_size], ydata[train_size:]

In [ ]:
# Window -> X timestep back
step_back= 1
X_train, Y_train = [], []
for i in range(len(x_train_dataset)-step_back - 1):
    a = x_train_dataset[i:(i+step_back), :]
    X_train.append(a)
    Y_train.append(y_train_dataset[i + step_back, 0])
X_train = np.array(X_train); Y_train = np.array(Y_train)
    
X_test, Y_test = [], []
for i in range(len(x_test_dataset)-step_back - 1):
    a = x_test_dataset[i:(i+step_back), :]
    X_test.append(a)
    Y_test.append(y_test_dataset[i + step_back, 0])
X_test = np.array(X_test); Y_test = np.array(Y_test)

print(X_train.shape); print(Y_train.shape);             
print(X_test.shape); print(Y_test.shape)

In [ ]:
# reshape input to be [samples, time steps, features]
    
X_train = np.reshape(X_train, (X_train.shape[0], step_back,6))
X_test  = np.reshape(X_test, (X_test.shape[0], step_back, 6))
Y_train = np.reshape(Y_train, (Y_train.shape[0], 1))
Y_test  = np.reshape(Y_test, (Y_test.shape[0], 1))

In [ ]:
# setup a GRU network in keras

output_size=7
activ_func="relu"
dropout=0.25
loss="mean_squared_error" 

model_gru = Sequential()
model_gru.add(GRU(75, return_sequences=True,input_shape=(X_train.shape[1], X_train.shape[2])))
model_gru.add(Dense(5))
model_gru.add(Dropout(0.25))
model_gru.add(GRU(units=30, return_sequences=True))
model_gru.add(Dense(units=output_size, activation="relu"))

model_gru.compile(loss='mean_squared_error', optimizer='adam')
model_gru.summary()

In [ ]:
gru_history = model_gru.fit(X_train, Y_train, epochs=70, batch_size=12, validation_data=(X_test, Y_test), shuffle=False, verbose=1)

In [ ]:
# Estimate model performance
trainScore = model_gru.evaluate(X_train, Y_train, verbose=1)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, np.sqrt(trainScore)))

In [ ]:
# Evaluate the skill of the Trained model
testPredict  = model_gru.predict(X_test)
testPredict.shape

In [ ]:
result1=[]
for i in range(0,testPredict.shape[0]):
    result1.append(testPredict[i][0])
    i=+1
result1

In [ ]:
# invert predictions
testPredict1 = scaler.inverse_transform(result1)

In [ ]:
result_pred=[]
for i in range(0,testPredict1.shape[0]):
    result_pred.append(testPredict1[i][0])
    i=+1

result_pred

In [ ]:
result11=[]
for i in range(0,len(result1)):
    result11.append(result1[i][0])
    i=+1
result11=np.reshape(result11,(-1,1))

In [ ]:
#rmse
math.sqrt(sum((result11-Y_test)**2)/6)

In [ ]:
#plt.figure(figsize=(10,6))
plt.scatter(result11,Y_test)

In [ ]:
# plot baseline and predictions
#plt.figure(figsize=(10,6))
plt.plot(result_pred)
plt.plot(label5[-6:].values)
plt.show()

# 5. Seocho-gu

In [ ]:
# normalize the dataset
scaler = MinMaxScaler()
xdata = scaler.fit_transform(feature)
ydata = scaler.fit_transform(label6)
wholedata=scaler.fit_transform(whole6)

In [ ]:
# Window -> X timestep back
step_back = 1
X_whole, Y_whole = [], []
for i in range(len(wholedata)-step_back - 1):
    a = wholedata[i:(i+step_back), 1:]
    X_whole.append(a)
    Y_whole.append(wholedata[i + step_back, 0])
X_whole = np.array(X_whole); Y_whole = np.array(Y_whole)
X_whole.shape

In [ ]:
from sklearn.model_selection import train_test_split,KFold,cross_val_score
import keras.backend as K 
import tensorflow as tf

kfold=KFold(10)

K.clear_session()

cvscores = []

for train, test in kfold.split(X_whole, Y_whole):

    X_train, X_test, y_train, y_test = train_test_split(X_whole, Y_whole, test_size = 0.1)
    X_train = np.reshape(X_train, (X_train.shape[0], step_back, 6))
    X_test  = np.reshape(X_test, (X_test.shape[0], step_back, 6))
    
    output_size=7
    activ_func="relu"
    dropout=0.25
    loss="mean_squared_error" 
    
    model = Sequential()
    model.add(GRU(75, return_sequences=True,input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(5))
    model.add(Dropout(dropout))
    model.add(GRU(units=30, return_sequences=True))
    model.add(Dense(units=output_size,activation=activ_func))

    model.compile(loss=loss, optimizer='adam',metrics =["mse"])

    model.fit(X_train, y_train, epochs=70, batch_size=12, verbose=0,shuffle=False)
    
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [ ]:
# split into train and test sets
train_size = int(len(xdata)*0.9)+4
x_train_dataset, x_test_dataset = xdata[0:train_size,:], xdata[train_size:,:]
y_train_dataset, y_test_dataset = ydata[0:train_size], ydata[train_size:]

In [ ]:
# Window -> X timestep back
step_back= 1
X_train, Y_train = [], []
for i in range(len(x_train_dataset)-step_back - 1):
    a = x_train_dataset[i:(i+step_back), :]
    X_train.append(a)
    Y_train.append(y_train_dataset[i + step_back, 0])
X_train = np.array(X_train); Y_train = np.array(Y_train)
    
X_test, Y_test = [], []
for i in range(len(x_test_dataset)-step_back - 1):
    a = x_test_dataset[i:(i+step_back), :]
    X_test.append(a)
    Y_test.append(y_test_dataset[i + step_back, 0])
X_test = np.array(X_test); Y_test = np.array(Y_test)

print(X_train.shape); print(Y_train.shape);             
print(X_test.shape); print(Y_test.shape)

In [ ]:
# reshape input to be [samples, time steps, features]
    
X_train = np.reshape(X_train, (X_train.shape[0], step_back, 6))
X_test  = np.reshape(X_test, (X_test.shape[0], step_back, 6))
Y_train = np.reshape(Y_train, (Y_train.shape[0], 1))
Y_test  = np.reshape(Y_test, (Y_test.shape[0], 1))

In [ ]:
# setup a GRU network in keras

output_size=7
activ_func="relu"
dropout=0.25
loss="mean_squared_error" 

model_gru = Sequential()
model_gru.add(GRU(75, return_sequences=True,input_shape=(X_train.shape[1], X_train.shape[2])))
model_gru.add(Dense(5))
model_gru.add(Dropout(0.25))
model_gru.add(GRU(units=30, return_sequences=True))
model_gru.add(Dense(units=output_size, activation="relu"))

model_gru.compile(loss='mean_squared_error', optimizer='adam')
model_gru.summary()

In [ ]:
gru_history = model_gru.fit(X_train, Y_train, epochs=70, batch_size=12, validation_data=(X_test, Y_test), shuffle=False, verbose=1)

In [ ]:
# Estimate model performance
trainScore = model_gru.evaluate(X_train, Y_train, verbose=1)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, np.sqrt(trainScore)))

In [ ]:
# Evaluate the skill of the Trained model
testPredict  = model_gru.predict(X_test)
testPredict.shape

In [ ]:
result1=[]
for i in range(0,testPredict.shape[0]):
    result1.append(testPredict[i][0])
    i=+1
result1

In [ ]:
# invert predictions
testPredict1 = scaler.inverse_transform(result1)

In [ ]:
result_pred=[]
for i in range(0,testPredict1.shape[0]):
    result_pred.append(testPredict1[i][0])
    i=+1

result_pred

In [ ]:
result11=[]
for i in range(0,len(result1)):
    result11.append(result1[i][0])
    i=+1
result11=np.reshape(result11,(-1,1))

In [ ]:
#rmse
math.sqrt(sum((result11-Y_test)**2)/6)

In [ ]:
#plt.figure(figsize=(10,6))
plt.scatter(result11,Y_test)

In [ ]:
# plot baseline and predictions
#plt.figure(figsize=(10,6))
plt.plot(result_pred)
plt.plot(label6[-6:].values)
plt.show()